# Replicate SAPLMA
In this notebook, we aim to replicate what the original paper has done, in order to have a working baseline.

Source: https://arxiv.org/pdf/2304.13734

## Imports, installations and declarations from previous notebooks

This section can be skipped and collapsed.

In [1]:
#@title Mount Drive, if needed, and check the HF_TOKEN is set and accessible
import os
from google.colab import drive, userdata

drive.mount('/content/drive')
DRIVE_PATH: str = '/content/drive/MyDrive/Final_Project/'
assert os.path.exists(DRIVE_PATH), 'Did you forget to create a shortcut in MyDrive named Final_Project this time as well? :('
%cd {DRIVE_PATH}
!ls

print()
assert userdata.get('HF_TOKEN'), 'Set up HuggingFace login secret properly in Colab!'
print('HF_TOKEN found')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1WdIP20OinXKeEN_xVOHEa6WVcY4eSO-k/Final_Project
 1_experiments_on_llama_and_saplma.ipynb   hallucination_detector
 2_replicate_saplma.ipynb		   publicDataset
'AML - First presentation.gslides'	   X_create_saplma_tensors_dataset.ipynb

HF_TOKEN found


In [2]:
%load_ext autoreload
%autoreload 1
%aimport hallucination_detector
import hallucination_detector

# Initialize Llama

In [3]:
import torch
from hallucination_detector.llama import LlamaInstruct
from hallucination_detector.dataset import StatementDataset
from hallucination_detector.extractor import LlamaHiddenStatesExtractor
from hallucination_detector.classifier import OriginalSAPLMAClassifier

torch.set_default_dtype(torch.float16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
llama = LlamaInstruct()
assert llama.device.type == 'cuda', 'The model should be running on a GPU. On CPU, it is impossible to run'

# Implement SAPLMA original model

In [5]:
dataset = StatementDataset.load_from_directory('publicDataset')
print(f'Found {len(dataset)} samples')

Loading file: cities_true_false.csv
Loading file: animals_true_false.csv
Loading file: elements_true_false.csv
Loading file: inventions_true_false.csv
Loading file: companies_true_false.csv
Loading file: generated_true_false.csv
Loading file: facts_true_false.csv
Found 6330 samples


In [6]:
original_saplma = OriginalSAPLMAClassifier()
original_saplma

OriginalSAPLMAClassifier(
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [7]:
hidden_extractor = LlamaHiddenStatesExtractor(llama)
hidden_extractor

## Train the SAPLMA classifier

According to the original paper, including the hidden layer index, and so on.

The only difference is that we are using llama **3.2** instead of version 2.